In [10]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [12]:
# =======================
# 0. Definir ruta base del proyecto
# =======================
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
ruta_gold = os.path.join(ruta_base, "GOLD")

In [14]:
df_top_10 = pd.read_parquet(os.path.join(ruta_gold, "top_10_campos_timeseries.parquet"))

In [16]:
# --- 1. Cargar los datos preparados de Maloob ---
campo_elegido = "MALOOB"
df_campo = df_top_10[df_top_10['campo'] == campo_elegido].copy()
df_campo['fecha'] = pd.to_datetime(df_campo['fecha'])
df_campo.set_index('fecha', inplace=True)
df_campo['produccion_mes_anterior'] = df_campo['petroleo_mbd'].shift(1)
df_campo['produccion_promedio_3m'] = df_campo['petroleo_mbd'].rolling(window=3).mean()
df_campo['mes'] = df_campo.index.month
df_campo['trimestre'] = df_campo.index.quarter


In [18]:
# --- 2. Preparación final para el modelo ---
# Eliminar filas con valores nulos generados por shift() y rolling()
df_modelo = df_campo.dropna()


In [20]:
# --- 2. Preparación final para el modelo ---
# Eliminar filas con valores nulos generados por shift() y rolling()
df_modelo = df_campo.dropna()

In [22]:
# Definir las 'pistas' (X) y lo que queremos predecir (y)
features = ['produccion_mes_anterior', 'produccion_promedio_3m', 'mes', 'trimestre']
target = 'petroleo_mbd'
X = df_modelo[features]
y = df_modelo[target]

In [24]:
# --- 3. Dividir los datos en Entrenamiento y Prueba ---
# Usaremos el 80% de los datos más antiguos para entrenar y el 20% más reciente para probar.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"\nDatos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")


Datos divididos: 85 para entrenamiento, 22 para prueba.


In [26]:
# --- 4. Entrenar el Modelo XGBoost ---
# XGBoost es un algoritmo de "Gradient Boosting" muy eficaz
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000, # Número de "árboles" que construye el modelo
    learning_rate=0.05, # Qué tan rápido aprende el modelo
    early_stopping_rounds=5 # Detener el entrenamiento si no mejora
)


In [28]:
# Entrenar el modelo
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)], # Usar los datos de prueba para ver el rendimiento
    verbose=False # No mostrar todos los pasos del entrenamiento
)

print("\nModelo entrenado exitosamente.")


Modelo entrenado exitosamente.


In [30]:
# --- 5. Evaluar el Modelo ---
# Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# Calcular el error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nEvaluación del modelo completada.")
print(f"Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Esto significa que, en promedio, las predicciones del modelo se equivocan en {rmse:.2f} mil barriles diarios.")



Evaluación del modelo completada.
Error Cuadrático Medio (RMSE): 4.77
Esto significa que, en promedio, las predicciones del modelo se equivocan en 4.77 mil barriles diarios.


In [32]:
# --- 4. Guardar el DataFrame con características en la capa GOLD ---

# Eliminar filas con valores nulos generados por shift() y rolling()
df_modelo = df_campo.dropna()

nombre_archivo_salida = "maloob_features_for_training.parquet"
ruta_completa_salida = os.path.join(ruta_gold, nombre_archivo_salida)

df_modelo.to_parquet(ruta_completa_salida)

print(f"\nDataFrame con características para Maloob guardado en:")
print(f"{ruta_completa_salida}")


DataFrame con características para Maloob guardado en:
C:\Users\elezr\Datalquemy\Proyectos\CNH\GOLD\maloob_features_for_training.parquet
